In [20]:
import pandas as pd
import ast
import networkx as nx
from tqdm import tqdm 

# --- read the CSV -----------------------------------------------------------
df = pd.read_csv('/Users/rezadoobary/Documents/ML-correlator/Graph_Edge_Data/den_graph_data_10.csv')

    # your file

# --- 2. Build & relabel graphs ---------------------------------------------
graphs = []
for edge_str in tqdm(df["EDGES"]):
    G = nx.Graph()
    G.add_edges_from(ast.literal_eval(edge_str))

    # >>> Relabel nodes to 0,1,2,…,n-1  (required by Graph2Vec) <<<
    G = nx.convert_node_labels_to_integers(G, ordering="sorted", label_attribute="orig_id")
    # the original label is stored in node attribute "orig_id" if you ever need it

    graphs.append(G)

100%|██████████| 153252/153252 [00:24<00:00, 6268.38it/s]


In [21]:
model = Graph2Vec(dimensions=1024, wl_iterations=5, epochs=50, workers=4)

In [22]:
from karateclub import Graph2Vec
import pickle
# --- 3. Train Graph2Vec -----------------------------------------------------

model.fit(graphs)
embeddings = model.get_embedding()      # shape: (n_graphs, 64)

# Save model
with open("/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/feature_embeddings/model_10.pkl", "wb") as f:
    pickle.dump(model, f)

# Save embeddings
with open("/Users/rezadoobary/Documents/ML-correlator/Tree classifier for graphs/mixed_loops/feature_embeddings/embeddings_10.pkl.pkl", "wb") as f:
    pickle.dump(embeddings, f)